In [14]:
import pandas as pd
import duckdb as db

from plotnine import *
pd.set_option('display.max_columns', None)

In [13]:
df = pd.read_parquet('../data/03_primary/core_applicants_jobs_prospects.parquet', )

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56539 entries, 0 to 56538
Data columns (total 32 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   vaga_id                       56539 non-null  int32 
 1   titulo_vaga                   56539 non-null  object
 2   cliente                       56539 non-null  object
 3   vaga_sap                      56539 non-null  int32 
 4   nivel_profissional_vaga       56539 non-null  object
 5   nivel_academico_vaga          56539 non-null  object
 6   nivel_ingles_vaga             56539 non-null  object
 7   nivel_espanhol_vaga           56539 non-null  object
 8   areas_atuacao                 56539 non-null  object
 9   principais_atividades         56539 non-null  object
 10  competencias                  56539 non-null  object
 11  estado                        56539 non-null  object
 12  cidade                        56539 non-null  object
 13  prospects       

### Verificando se o JOIN está correto. 

> Utilização: Por exemplo, a vaga 10976 (chave no Jobs.json), possui 25 prospecções (chave 10976 no prospects.json), onde o candidato “Sr. Thales Freitas”  (chave 41496 no applicants.json) foi contratado.

In [16]:
df.query("vaga_id == 10976")

,vaga_id,titulo_vaga,cliente,vaga_sap,nivel_profissional_vaga,nivel_academico_vaga,nivel_ingles_vaga,nivel_espanhol_vaga,areas_atuacao,principais_atividades,competencias,estado,cidade,prospects,codigo,nome_candidato,situacao_candidado,data_candidatura,ultima_atualizacao,comentario,recrutador,prospect_codigo,nome,email,area_atuacao,nivel_profissional_candidato,nivel_academico_candidato,nivel_ingles_candidato,nivel_espanhol_candidato,conhecimentos_tecnicos,cv_pt,target_contratado
41394,10976,Analista NOC,Porter-Wilson,0,Analista,Ensino Superior Completo,Básico,Básico,TI - Suporte-,1 analista de N1 NOC para trabalhar de terça a...,1 analista de N1 NOC para trabalhar de terça a...,São Paulo,São Paulo,10976,41531,Sra. Beatriz Teixeira,Inscrito,03-02-2023,,,Caroline Machado,41531,Sra. Beatriz Teixeira,sra._beatriz_teixeira@gmail.com,Gestão e Alocação de Recursos de TI,,Pós Graduação Cursando,Intermediário,Nenhum,,"\nbrasileira, solteira, 23 anos.\ncampinas, sã...",0
41395,10976,Analista NOC,Porter-Wilson,0,Analista,Ensino Superior Completo,Básico,Básico,TI - Suporte-,1 analista de N1 NOC para trabalhar de terça a...,1 analista de N1 NOC para trabalhar de terça a...,São Paulo,São Paulo,10976,41527,Sr. Breno Oliveira,Prospect,03-02-2023,03-02-2023,,Melina Montenegro,41527,Sr. Breno Oliveira,sr._breno_oliveira@hotmail.com,TI - Suporte,,Ensino Superior Completo,Básico,Básico,,experiência\nletiel\nmitsuo analista noc n2/ho...,0
41396,10976,Analista NOC,Porter-Wilson,0,Analista,Ensino Superior Completo,Básico,Básico,TI - Suporte-,1 analista de N1 NOC para trabalhar de terça a...,1 analista de N1 NOC para trabalhar de terça a...,São Paulo,São Paulo,10976,41507,Dr. Brayan Rezende,Prospect,02-02-2023,02-02-2023,,Maria Clara Pires,41507,Dr. Brayan Rezende,dr._brayan_rezende@gmail.com,,,,,,,resumo\niniciei a minha trajetória profissiona...,0
41397,10976,Analista NOC,Porter-Wilson,0,Analista,Ensino Superior Completo,Básico,Básico,TI - Suporte-,1 analista de N1 NOC para trabalhar de terça a...,1 analista de N1 NOC para trabalhar de terça a...,São Paulo,São Paulo,10976,41506,Dr. Theo Fernandes,Prospect,02-02-2023,02-02-2023,,Maria Clara Pires,41506,Dr. Theo Fernandes,dr._theo_fernandes@gmail.com,,,,,,,"resumo\nobjetivos profissionais\ninformática, ...",0
41398,10976,Analista NOC,Porter-Wilson,0,Analista,Ensino Superior Completo,Básico,Básico,TI - Suporte-,1 analista de N1 NOC para trabalhar de terça a...,1 analista de N1 NOC para trabalhar de terça a...,São Paulo,São Paulo,10976,41502,Dr. Yan Vargas,Prospect,02-02-2023,02-02-2023,,Maria Clara Pires,41502,Dr. Yan Vargas,dr._yan_vargas@hotmail.com,,,,,,,\n\nanalista de monitoramento\n\ncódigo apinfo...,0
41399,10976,Analista NOC,Porter-Wilson,0,Analista,Ensino Superior Completo,Básico,Básico,TI - Suporte-,1 analista de N1 NOC para trabalhar de terça a...,1 analista de N1 NOC para trabalhar de terça a...,São Paulo,São Paulo,10976,41501,Sr. Bruno Rocha,Prospect,02-02-2023,02-02-2023,,Maria Clara Pires,41501,Sr. Bruno Rocha,sr._bruno_rocha@gmail.com,,,,,,,"resumo\nobjetivos profissionais\ninformática, ...",0
41400,10976,Analista NOC,Porter-Wilson,0,Analista,Ensino Superior Completo,Básico,Básico,TI - Suporte-,1 analista de N1 NOC para trabalhar de terça a...,1 analista de N1 NOC para trabalhar de terça a...,São Paulo,São Paulo,10976,41498,Sr. João Viana,Encaminhado ao Requisitante,02-02-2023,02-02-2023,,Melina Montenegro,41498,Sr. João Viana,sr._joão_viana@hotmail.com,"TI - Qualidade/Testes, TI - Suporte, TI - Telecom",,Ensino Técnico Cursando,Intermediário,Avançado,,nascimento: 13/05/1992\nbairro: fazenda da jut...,0
41401,10976,Analista NOC,Porter-Wilson,0,Analista,Ensino Superior Completo,Básico,Básico,TI - Suporte-,1 analista de N1 NOC para trabalhar de terça a...,1 analista de N1 NOC para trabalhar de terça a...,São Paulo,São Paulo,10976,41496,Sr. Thales Freitas,Contratado pela Decision,02-02-2023,08-02-2023,,Melina Montenegro,41496,Sr. Thales Freitas,sr._thales_freitas@gmail.com,,,,,,,\n\n\ncont

# Verificando os tipos de situação de candidatos


In [8]:
df.situacao_candidado.value_counts()

situacao_candidado
Prospect                          20005
Encaminhado ao Requisitante       16120
Inscrito                           3978
Não Aprovado pelo Cliente          3489
Contratado pela Decision           2757
Desistiu                           2349
Não Aprovado pelo RH               1765
Não Aprovado pelo Requisitante      765
Entrevista Técnica                  579
Sem interesse nesta vaga            576
Entrevista com Cliente              469
Em avaliação pelo RH                375
Contratado como Hunting             226
Aprovado                            209
Desistiu da Contratação              59
Documentação PJ                       4
Documentação CLT                      3
Recusado                              2
Documentação Cooperado                2
Encaminhar Proposta                   2
Proposta Aceita                       1
Name: count, dtype: int64

In [38]:
import json
import logging
from typing import Any, Dict

import numpy as np
import pandas as pd

logger = logging.getLogger(__name__)

def create_ranking_system() -> Dict[str, int]:
    """
    Alternative ranking system using only positive scores (0-10 scale).
    Some algorithms prefer non-negative rankings.

    Returns:
        Dictionary mapping status to ranking score (0-10)
    """
    ranking_system = {
        # TIER 1: Success
        'Contratado pela Decision': 10,
        'Contratado como Hunting': 9,
        'Proposta Aceita': 8,

        # TIER 2: Final Stages of hiring process
        'Aprovado': 7,
        'Documentação PJ': 6,
        'Documentação CLT': 6,
        'Documentação Cooperado': 6,
        'Encaminhar Proposta': 5,

        # TIER 3: Advanced stages
        'Entrevista com Cliente': 4,
        'Entrevista Técnica': 4,
        'Encaminhado ao Requisitante': 3,

        # TIER 4: Early stages
        'Em avaliação pelo RH': 2,
        'Inscrito': 2,

        # TIER 5: Bad outcomes
        'Desistiu da Contratação': 1,
        'Desistiu': 1,
        'Sem interesse nesta vaga': 1,

        # TIER 6: Rejections
        'Não Aprovado pelo Cliente': 0,
        'Não Aprovado pelo RH': 0,
        'Não Aprovado pelo Requisitante': 0,
        'Recusado': 0
    }

    return ranking_system

def dataset_read(parquet_path: str) -> pd.DataFrame:
    """
    Read a Parquet dataset.

    Args:
        parquet_path: Path to the Parquet file

    Returns:
        DataFrame containing the dataset
    """
    try:
        df = pd.read_parquet(parquet_path)
        logger.info(f"Dataset loaded successfully from {parquet_path}")
        return df
    except Exception as e:
        logger.error(f"Failed to read dataset from {parquet_path}: {e}")
        raise

def apply_ranking_to_prospects(
    prospects_df: pd.DataFrame, 
    ranking_system: Dict[str, int] = None,
    status_column: str = 'situacao_candidado'
) -> pd.DataFrame:
    """
    Apply ranking scores to prospects DataFrame.
    
    Args:
        prospects_df: DataFrame with prospect data
        ranking_system: Dictionary mapping status to scores
        status_column: Column name containing the status
        
    Returns:
        DataFrame with added ranking column
    """
    if ranking_system is None:
        ranking_system = create_ranking_system()  # Fixed function name
    
    # Create a copy to avoid modifying original
    df = prospects_df.copy()
    
    # Apply ranking
    df['ranking_score'] = df[status_column].map(ranking_system)
    
    # Handle unmapped statuses
    unmapped_statuses = df[df['ranking_score'].isna()][status_column].unique()
    if len(unmapped_statuses) > 0:
        logger.warning(f"Unmapped statuses found: {unmapped_statuses}")
        # Assign neutral score to unmapped statuses
        df['ranking_score'] = df['ranking_score'].fillna(0).astype(int)
    
    # Log ranking distribution
    ranking_dist = df['ranking_score'].value_counts().sort_index()
    logger.info(f"Ranking score distribution:\n{ranking_dist}")
    
    return df    

# Main execution
if __name__ == "__main__":
    # Load the dataset
    prospects_df = dataset_read(parquet_path="../data/03_primary/core_applicants_jobs_prospects.parquet")
    
    # Create ranking system
    ranking_system = create_ranking_system()
    
    # Apply ranking to prospects
    prospects_ranked = apply_ranking_to_prospects(prospects_df, ranking_system)
    
    # Print results
    print(prospects_ranked.head())

Unmapped statuses found: ['Prospect' None]


   vaga_id                                        titulo_vaga  \
0     4530                                CONSULTOR CONTROL M   
1     4530                                CONSULTOR CONTROL M   
2     4531  2021-2607395-PeopleSoft Application Engine-Dom...   
3     4531  2021-2607395-PeopleSoft Application Engine-Dom...   
4     4533  2021-2605708-Microfocus Application Life Cycle...   

                    cliente  vaga_sap nivel_profissional_vaga  \
0  Morris, Moran and Dodson         0                   Pleno   
1  Morris, Moran and Dodson         0                   Pleno   
2         Gonzalez and Sons         0                  Sênior   
3         Gonzalez and Sons         0                  Sênior   
4              Barnes-Woods         0                  Sênior   

       nivel_academico_vaga nivel_ingles_vaga nivel_espanhol_vaga  \
0  Ensino Superior Completo            Nenhum              Nenhum   
1  Ensino Superior Completo            Nenhum              Nenhum   
2     Ensin

In [40]:
from sklearn.preprocessing import OrdinalEncoder


hierarchy_order = [
    'Aprendiz', 'Trainee', 'Auxiliar', 'Assistente', 'Técnico de Nível Médio',
    'Júnior', 'Analista', 'Pleno', 'Sênior', 'Especialista', 'Coordenador',
    'Supervisor', 'Líder', 'Gerente'
]

# Using sklearn OrdinalEncoder
ordinal_encoder = OrdinalEncoder(categories=[hierarchy_order])
df_ordinal = prospects_ranked.copy()
df_ordinal['vaga_nivel_encoded'] = ordinal_encoder.fit_transform(df_ordinal[['nivel_profissional_vaga']]).astype(int)

In [41]:
for col in df_ordinal.select_dtypes(include=['object']).columns:



,vaga_id,titulo_vaga,cliente,vaga_sap,nivel_profissional_vaga,nivel_academico_vaga,nivel_ingles_vaga,nivel_espanhol_vaga,areas_atuacao,principais_atividades,competencias,estado,cidade,prospects,codigo,nome_candidato,situacao_candidado,data_candidatura,ultima_atualizacao,comentario,recrutador,prospect_codigo,nome,email,area_atuacao,nivel_profissional_candidato,nivel_academico_candidato,nivel_ingles_candidato,nivel_espanhol_candidato,conhecimentos_tecnicos,cv_pt,target_contratado,ranking_score,vaga_nivel_encoded
0,4530,CONSULTOR CONTROL M,"Morris, Moran and Dodson",0,Pleno,Ensino Superior Completo,Nenhum,Nenhum,TI - Desenvolvimento/Programação-,- Experiência comprovada em projetos de control-M,- Experiência comprovada em projetos de control-M,Rio de Janeiro,Rio de Janeiro,4530,25632,José Vieira,Encaminhado ao Requisitante,25-03-2021,25-03-2021,"Encaminhado para - PJ R$ 72,00/hora",Ana Lívia Moreira,25632,José Vieira,josé_vieira@gmail.com,,,,,,,\ndados pessoais\nestado civil: casado\nidade:...,0,3,7
1,4530,CONSULTOR CONTROL M,"Morris, Moran and Dodson",0,Pleno,Ensino Superior Completo,Nenhum,Nenhum,TI - Desenvolvimento/Programação-,- Experiência comprovada em projetos de control-M,- Experiência comprovada em projetos de control-M,Rio de Janeiro,Rio de Janeiro,4530,25529,Srta. Isabela Cavalcante,Encaminhado ao Requisitante,22-03-2021,23-03-2021,"encaminhado para - R$ 6.000,00 – CLT Full , n...",Ana Lívia Moreira,25529,Srta. Isabela Cavalcante,srta._isabela_cavalcante@hotmail.com,"TI - Governança, TI - Infraestrutura, TI - Pro...",,Ensino Superior Completo,Intermediário,Básico,,"solteiro, 47 anos\n\nestrada meringuava, nº 17...",0,3,7
2,4531,2021-2607395-PeopleSoft Application Engine-Dom...,Gonzalez and Sons,0,Sênior,Ensino Médio Completo,Nenhum,Nenhum,Gestão e Alocação de Recursos de TI-,Key skills required for the job are:\n\nPeople...,O recurso Peoplesoft tem como responsabilidade...,São Paulo,São Paulo,4531,25364,Sra. Yasmin Fernandes,Contratado pela Decision,17-03-2021,12-04-2021,Data de Inicio: 12/04/2021,Juliana Cassiano,25364,Sra. Yasmin Fernandes,sra._yasmin_fernandes@hotmail.com,TI - Projetos,,Ensino Superior Completo,Avançado,Intermediário,"- PeopleSoft (PeopleTools 8.49, 8.53, 8.55, 8....",\n\nárea de atuação: lider de consultoria / ge...,1,10,8
3,4531,2021-2607395-PeopleSoft Application Engine-Dom...,Gonzalez and Sons,0,Sênior,Ensino Médio Completo,Nenhum,Nenhum,Gestão e Alocação de Recursos de TI-,Key skills required for the job are:\n\nPeople...,O recurso Peoplesoft tem como responsabilidade...,São Paulo,São Paulo,4531,25360,Alexia Barbosa,Encaminhado ao Requisitante,17-03-2021,17-03-2021,,Juliana Cassiano,25360,Alexia Barbosa,alexia_barbosa@hotmail.com,,,,,,,informações pessoais\n estado civil: casado\n...,0,3,8
4,4533,2021-2605708-Microfocus Application Life Cycle...,Barnes-Woods,0,Sênior,Ensino Médio Completo,Técnico,Fluente,Gestão e Alocação de Recursos de TI-,Arquiteto\n\nFoco na área e automação.\n\nRequ...,Arquiteto\n\nFoco na área e automação.\n\nRequ...,São Paulo,São Paulo,4533,26338,Arthur Almeida,Contratado pela Decision,29-04-2021,18-05-2021,,Stella Vieira,26338,Arthur Almeida,arthur_almeida@gmail.com,,,,,,,"solteiro, brasileiro, 21/06/1987\nhabilitação ...",1,10,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56534,13073,Feature Store - 14283623,"Wade, Bernard and Bailey",0,Analista,Ensino Superior Completo,Básico,Básico,TI - Projetos-,Profissional de visualização de dados.\nskills...,Hibrido/Flexível São Paulo\nNão requer idioma ...,São Paulo,São Paulo,<NA>,<NA>,None,None,None,None,None,None,<NA>,None,None,None,None,None,None,None,None,None,0,0,6
56535,13005,RPA PEGA ROBOTICS Pleno ou Sênior - 839,Bell Group,0,Pleno,Ensino Médio Completo,Fluente,Básico,TI - Projetos-,Experiência comprovada na instalação e desenvo...,Nível Profissional: Consultor / Desenvolvedor ...,São Paulo,São Paulo,<NA>,<NA>,None,None,None,

In [56]:
object_columns = df_ordinal.select_dtypes(include=['object']).columns

# Replace empty strings and whitespace-only strings with NaN for all object columns
for col in object_columns:
    df_ordinal[col] = df_ordinal[col].replace(r'^\s*$', None, regex=True)
    df_ordinal[col] = df_ordinal[col].replace('', None)

In [65]:
for col in df_ordinal.columns:
    print(f"Column: {col} \n---- Unique Values: {df_ordinal[col].nunique()}, Nulls: {df_ordinal[col].isna().sum()}")


Column: vaga_id 
---- Unique Values: 14081, Nulls: 0
Column: titulo_vaga 
---- Unique Values: 12452, Nulls: 0
Column: cliente 
---- Unique Values: 112, Nulls: 0
Column: vaga_sap 
---- Unique Values: 2, Nulls: 0
Column: nivel_profissional_vaga 
---- Unique Values: 14, Nulls: 0
Column: nivel_academico_vaga 
---- Unique Values: 16, Nulls: 0
Column: nivel_ingles_vaga 
---- Unique Values: 6, Nulls: 0
Column: nivel_espanhol_vaga 
---- Unique Values: 6, Nulls: 5007
Column: areas_atuacao 
---- Unique Values: 73, Nulls: 22
Column: principais_atividades 
---- Unique Values: 11709, Nulls: 8
Column: competencias 
---- Unique Values: 11551, Nulls: 15
Column: estado 
---- Unique Values: 26, Nulls: 45
Column: cidade 
---- Unique Values: 166, Nulls: 207
Column: prospects 
---- Unique Values: 11277, Nulls: 2804
Column: codigo 
---- Unique Values: 29399, Nulls: 2804
Column: nome_candidato 
---- Unique Values: 26241, Nulls: 2804
Column: situacao_candidado 
---- Unique Values: 21, Nulls: 2804
Column: data

In [74]:
import seaborn as sns